## Vertical photos combining

In [1]:
import random
from usefull_functions import total_score, create_vertical_slides, load_photos_from_json

In [4]:
v_photos = load_photos_from_json("../data/vertical_photos.json")
# v_photos = v_photos[:10000]

### Random Stitching

In [5]:
def random_pair_vertical_photos(vertical_photos):
    """
    Returns list of random (photo1, photo2) pairs.
    """

    photos = vertical_photos.copy()

    if len(photos) % 2 == 1:
        photos = photos[:-1]

    random.shuffle(photos)

    pairs = []
    for i in range(0, len(photos), 2):
        pairs.append((photos[i], photos[i + 1]))

    return pairs


In [7]:
v_slides = create_vertical_slides(random_pair_vertical_photos, v_photos)
print(v_slides[:10])

[{'photos': [77504, 70234], 'tags': {'tc', 't01', 'tq1', 't31', 'tb6', 'tq6', 't82', 't34', 'th4', 'tf4', 'tl5'}}, {'photos': [81224, 37000], 'tags': {'t53', 't01', 't52', 't94', 'th5', 'tk3', 'tr6', 't34', 'th4', 'tz2', 'tz5'}}, {'photos': [78320, 53871], 'tags': {'tv4', 't6', 't94', 'tp2', 'th5', 'tz6', 'tx2', 'tz2', 'ts2', 'tn4', 'tl5', 't86', 'tp1', 'td1', 't47', 't05', 'th4', 'tc5'}}, {'photos': [28929, 15102], 'tags': {'t91', 't53', 'th5', 'tk1', 'tw3', 'tq2', 'tp6', 'tj4', 'th4', 'tn6'}}, {'photos': [144, 36414], 'tags': {'t01', 'td4', 'tw5', 'tw3', 't34', 't85', 'th4', 'tx2', 'tn6', 'tb7'}}, {'photos': [2030, 52602], 'tags': {'t87', 'tc1', 'tp2', 'th5', 'tw5', 'tx1', 'tq2', 'tz', 't82', 'tp6', 'tf2', 'ts2', 'tc4', 'tl5', 't86', 'tk5', 'td6', 'tn3', 'tc5', 'tf4', 'tb7'}}, {'photos': [70250, 39591], 'tags': {'t6', 't01', 'tx1', 'tk1', 'tz6', 'ts2', 't85', 'tl5', 't52', 't9', 'tl3', 'tp1', 'td4', 'tf', 'tw3', 'tr5', 'th4', 'tf4'}}, {'photos': [3868, 77345], 'tags': {'t6', 'tv4', '

In [8]:
score = total_score(v_slides)
print(score)

146603


### Similar photos

In [1]:
def similar_pair_vertical_photos(vertical_photos, k=200):
    """
    A fast greedy implementation for large datasets.
    k: number of candidates based on the number of tags for each photo
    """
    photos = sorted(vertical_photos, key=lambda x: len(x["tags"]))
    if len(photos) % 2 == 1:
        photos = photos[:-1]

    used = set()
    pairs = []
    n = len(photos)

    for idx, photo in enumerate(photos):
        if photo["id"] in used:
            continue

        start = max(0, idx - k)
        end = min(n, idx + k)
        best_score = -1
        best_pair = None

        for j in range(start, end):
            candidate = photos[j]
            if candidate["id"] in used or candidate["id"] == photo["id"]:
                continue
            score = len(photo["tags"] & candidate["tags"])
            if score > best_score:
                best_score = score
                best_pair = candidate

        if best_pair is not None:
            pairs.append((photo, best_pair))
            used.add(photo["id"])
            used.add(best_pair["id"])

    return pairs


In [9]:
v_slides = create_vertical_slides(similar_pair_vertical_photos, v_photos)
print(v_slides[:10])

[{'photos': [805, 4053], 'tags': {'t34', 'tz2', 't52'}}, {'photos': [991, 3078], 'tags': {'td4', 't47', 'tw3'}}, {'photos': [1060, 71525], 'tags': {'t53', 'ts2'}}, {'photos': [1740, 7716], 'tags': {'t01', 'tq2', 'tr5'}}, {'photos': [2259, 14328], 'tags': {'tx', 'th5', 'tz4'}}, {'photos': [2800, 23469], 'tags': {'tm3', 'tj1', 't05'}}, {'photos': [4069, 10305], 'tags': {'t31', 'tm2', 'tk5'}}, {'photos': [4567, 25080], 'tags': {'t53', 't85'}}, {'photos': [6215, 29216], 'tags': {'tq1', 't47', 'tr5'}}, {'photos': [7411, 761], 'tags': {'th6', 'th4', 'tm'}}]


In [10]:
score = total_score(v_slides)
print(score)

113265


### Different photos

In [3]:
def different_pair_vertical_photos(vertical_photos, k=300):
    """
    Combines photos with minimal tag intersection.
    k: Number of candidates for finding the minimal intersection
    """
    photos = sorted(vertical_photos, key=lambda x: len(x["tags"]))
    if len(photos) % 2 == 1:
        photos = photos[:-1]

    used = set()
    pairs = []
    n = len(photos)

    for idx, photo in enumerate(photos):
        if photo["id"] in used:
            continue

        start = max(0, idx - k)
        end = min(n, idx + k)
        best_score = float('inf')
        best_pair = None

        for j in range(start, end):
            candidate = photos[j]
            if candidate["id"] in used or candidate["id"] == photo["id"]:
                continue
            score = len(photo["tags"] & candidate["tags"])
            if score < best_score:
                best_score = score
                best_pair = candidate

        if best_pair is not None:
            pairs.append((photo, best_pair))
            used.add(photo["id"])
            used.add(best_pair["id"])

    return pairs

In [4]:
v_slides = create_vertical_slides(different_pair_vertical_photos, v_photos)
print(v_slides[:10])

[{'photos': [805, 991], 'tags': {'tz2', 't52', 'td4', 't47'}}, {'photos': [1060, 1740], 'tags': {'ts2', 't01', 'tr5', 't53'}}, {'photos': [2259, 2800], 'tags': {'tz4', 'tj1', 'th5', 't05'}}, {'photos': [3078, 4053], 'tags': {'t34', 'tw3', 'td4', 'tz2'}}, {'photos': [4069, 4567], 'tags': {'tm2', 't31', 't53', 't85'}}, {'photos': [6215, 7411], 'tags': {'t47', 'tq1', 'th6', 'th4'}}, {'photos': [7716, 8528], 'tags': {'t87', 't57', 'tq2', 'tr5'}}, {'photos': [9251, 10305], 'tags': {'tn6', 'tw3', 'tm2', 'tk5'}}, {'photos': [10565, 11256], 'tags': {'tc4', 'tz2', 'tq6', 't87'}}, {'photos': [11702, 13751], 'tags': {'t87', 'tz2', 'th6', 'tl3'}}]


In [5]:
score = total_score(v_slides)
print(score)

209063
